In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import os
print(tf.__version__)

2.3.0


In [11]:
data_dir = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

In [12]:
pixels =224
BATCH_SIZE = 32 
IMAGE_SIZE = (pixels, pixels)
NUM_CLASSES = 5

In [13]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
                   interpolation="bilinear")

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset="validation", shuffle=False, **dataflow_kwargs)

train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory(
    data_dir, subset="training", shuffle=True, **dataflow_kwargs)

Found 731 images belonging to 5 classes.
Found 2939 images belonging to 5 classes.


In [14]:
labels_idx = (train_generator.class_indices)
idx_labels = dict((v,k) for k,v in labels_idx.items())

In [15]:
idx_labels

{0: 'daisy', 1: 'dandelion', 2: 'roses', 3: 'sunflowers', 4: 'tulips'}

In [16]:
os.environ["TFHUB_CACHE_DIR"] = '../chapter_07/imagenet_resnet_v2_50_feature_vector_4'

mdl = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4",
                   trainable=False),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', name = 'custom_class')
])
mdl.build([None, 224, 224, 3])

In [17]:
mdl.compile(
  optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

In [18]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
mdl.fit(
    train_generator,
    epochs=5, steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps)

Epoch 1/5
91/91 [==============================] - 97s 1s/step - loss: 1.2773 - accuracy: 0.6797 - val_loss: 1.1753 - val_accuracy: 0.8054
Epoch 2/5
91/91 [==============================] - 97s 1s/step - loss: 1.1341 - accuracy: 0.8490 - val_loss: 1.1471 - val_accuracy: 0.8395
Epoch 3/5
91/91 [==============================] - 99s 1s/step - loss: 1.1092 - accuracy: 0.8806 - val_loss: 1.1292 - val_accuracy: 0.8523
Epoch 4/5
91/91 [==============================] - 98s 1s/step - loss: 1.0924 - accuracy: 0.9020 - val_loss: 1.1278 - val_accuracy: 0.8523
Epoch 5/5
91/91 [==============================] - 98s 1s/step - loss: 1.0817 - accuracy: 0.9133 - val_loss: 1.1225 - val_accuracy: 0.8537
